# Simulation

**pyecsca** is able to simulate computation of key generation, ECDH and ECDSA while tracing particular actions performed by the implementation as well as intermediate values. These traces are collected by the context (see the `Context` and `DefaultContext` classes). There is always one context active. For performance reasons, by default it is a `NullContext` instance, which does not trace anything.

These traces are useful for attacks which rely on computing particular intermediate values during the ECC computation.


## Initialisation

In [1]:
from binascii import hexlify
from copy import copy

from pyecsca.ec.key_generation import KeyGeneration
from pyecsca.ec.key_agreement import ECDH_SHA1
from pyecsca.ec.signature import ECDSA_SHA1
from pyecsca.ec.params import get_params
from pyecsca.ec.mult import LTRMultiplier
from pyecsca.ec.context import DefaultContext, local

In [2]:
p256 = get_params("secg", "secp256r1", "projective")
model = p256.curve.model
coords = p256.curve.coordinate_model

add = coords.formulas["add-2007-bl"]
dbl = coords.formulas["dbl-2007-bl"]

mult = LTRMultiplier(add, dbl, None)

## Key generation

In [3]:
keygen = KeyGeneration(copy(mult), p256, False)

In [4]:
with local(DefaultContext()) as keygen_ctx:
    private, public = keygen.generate()
print(private)
print(public)
print(keygen_ctx.actions)

27534349110771347155923075045818046165979888998818651586000312719216910833318
[X=83933898411588104793089377774707107506941266646933982525624598759013717245621, Y=26527425049573216561448628629620140099790717296630687777633119014160647664021, Z=64088794318210508120032658643489914503426087501574872600975980808063169111279]
KeygenAction(DomainParameters(secg/secp256r1))
	RandomModAction(ffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551)
	ScalarMultiplicationAction([X=48439561293906451759052585252797914202762949526041747995844080717082404635286, Y=36134250956749795798585127919587881956611106672985015071877198253568414405109, Z=1], 27534349110771347155923075045818046165979888998818651586000312719216910833318)
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(add[add-2007-bl])
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(add[add-2007-bl])
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(add[add-2007-bl])
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(dbl[dbl-2007-bl])


## Signing

In [5]:
ecdsa = ECDSA_SHA1(copy(mult), p256, add, public, private)
data = b"something"

In [6]:
with local(DefaultContext()) as sign_ctx:
    signature = ecdsa.sign_data(data)
print(data)
print(signature)
print(sign_ctx.actions)

b'something'
(r=94431243472779457770181078394757042074208310087471629129794997149179235279505, s=89619810655289534561305906670069498479061576363695717048711157188366011214888)
ECDSASignAction(DomainParameters(secg/secp256r1), <built-in function openssl_sha1>, b'something', 27534349110771347155923075045818046165979888998818651586000312719216910833318)
	RandomModAction(ffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551)
	ScalarMultiplicationAction([X=48439561293906451759052585252797914202762949526041747995844080717082404635286, Y=36134250956749795798585127919587881956611106672985015071877198253568414405109, Z=1], 16976072681883240233350541036004334878693289354054036605340296754667307900719)
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(add[add-2007-bl])
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(add[add-2007-bl])
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(

In [7]:
with local(DefaultContext()) as verify_ctx:
    verified = ecdsa.verify_data(signature, data)
print(verified)
print(verify_ctx.actions)

True
ECDSAVerifyAction(DomainParameters(secg/secp256r1), <built-in function openssl_sha1>, b'something', (r=94431243472779457770181078394757042074208310087471629129794997149179235279505, s=89619810655289534561305906670069498479061576363695717048711157188366011214888), [X=83933898411588104793089377774707107506941266646933982525624598759013717245621, Y=26527425049573216561448628629620140099790717296630687777633119014160647664021, Z=64088794318210508120032658643489914503426087501574872600975980808063169111279])
	ScalarMultiplicationAction([X=48439561293906451759052585252797914202762949526041747995844080717082404635286, Y=36134250956749795798585127919587881956611106672985015071877198253568414405109, Z=1], 111701592732851471232423665291050553644989131547247713497227113418554581357191)
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(add[add-2007-bl])
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(add[add-2007-bl])
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(add[add-2007-bl])
		For

## Key agreement

In [8]:
other_private, other_public = keygen.generate()

ecdh_a = ECDH_SHA1(copy(mult), p256, public, other_private)
ecdh_b = ECDH_SHA1(copy(mult), p256, other_public, private)
with local(DefaultContext()) as ecdh_ctx:
    ecdh_a_result = ecdh_a.perform()
    ecdh_b_result = ecdh_b.perform()
print(hexlify(ecdh_a_result))
print(hexlify(ecdh_b_result))
print(ecdh_a_result == ecdh_b_result)
print(ecdh_ctx.actions)

b'2306c549dfd8ff5a3315058491f05553af5e9af3'
b'2306c549dfd8ff5a3315058491f05553af5e9af3'
True
ECDHAction(DomainParameters(secg/secp256r1), <built-in function openssl_sha1>, 103438430940984774541828939544194320318457161320409005542664649093940739658887, [X=83933898411588104793089377774707107506941266646933982525624598759013717245621, Y=26527425049573216561448628629620140099790717296630687777633119014160647664021, Z=64088794318210508120032658643489914503426087501574872600975980808063169111279])
	ScalarMultiplicationAction([X=83933898411588104793089377774707107506941266646933982525624598759013717245621, Y=26527425049573216561448628629620140099790717296630687777633119014160647664021, Z=64088794318210508120032658643489914503426087501574872600975980808063169111279], 103438430940984774541828939544194320318457161320409005542664649093940739658887)
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(add[add-2007-bl])
		FormulaAction(dbl[dbl-2007-bl])
		FormulaAction(add[add-2007-bl])
		FormulaActio

## Walking the trace

As visible from the outputs above, `DefaultContext` traces the actions performed by the implementation in an ordered tree where the child relationship means that some actions happened during another action and order of children gives the order of operations. In the above example of ECDH, two ECDH executions are visible, each consisting of one scalar multuplication which consists of several applications of `add` and `dbl` formulas.

We can examine this tree in the first ECDH execution and see that the scalar multiplier used was not setup to be regular (see `LTRMultiplier` argument `always`) and that the order of operations leaks bits of the scalar. This fact will be easily exploitable on a power trace via SPA.

In [9]:
tree = ecdh_ctx.actions

ecdh_action, subtree = tree.get_by_index([0])

scalarmult_action, subtree = subtree.get_by_index([0])
recovered_private = 1
for formula_call in subtree:
    if formula_call.formula.shortname == "add":
        recovered_private |= 1
    elif formula_call.formula.shortname == "dbl":
        recovered_private <<= 1
    print(formula_call.formula.shortname)

print(bin(int(other_private)))
print(bin(int(recovered_private)))
print(other_private == recovered_private)

dbl
add
dbl
add
dbl
add
dbl
dbl
add
dbl
add
dbl
add
dbl
dbl
add
dbl
add
dbl
dbl
add
dbl
dbl
dbl
add
dbl
add
dbl
dbl
add
dbl
dbl
dbl
add
dbl
dbl
dbl
add
dbl
add
dbl
add
dbl
dbl
add
dbl
dbl
dbl
add
dbl
dbl
dbl
dbl
dbl
dbl
dbl
add
dbl
add
dbl
add
dbl
add
dbl
dbl
dbl
add
dbl
dbl
dbl
dbl
add
dbl
dbl
dbl
dbl
add
dbl
dbl
dbl
add
dbl
add
dbl
dbl
add
dbl
dbl
add
dbl
dbl
add
dbl
add
dbl
add
dbl
add
dbl
dbl
add
dbl
add
dbl
dbl
add
dbl
add
dbl
add
dbl
dbl
dbl
add
dbl
dbl
add
dbl
dbl
dbl
add
dbl
dbl
add
dbl
dbl
add
dbl
dbl
add
dbl
add
dbl
add
dbl
dbl
dbl
add
dbl
add
dbl
add
dbl
dbl
add
dbl
dbl
add
dbl
add
dbl
dbl
dbl
add
dbl
dbl
add
dbl
dbl
dbl
add
dbl
dbl
add
dbl
dbl
dbl
add
dbl
dbl
add
dbl
dbl
add
dbl
add
dbl
add
dbl
dbl
dbl
dbl
dbl
add
dbl
dbl
add
dbl
add
dbl
add
dbl
dbl
dbl
add
dbl
dbl
add
dbl
add
dbl
dbl
add
dbl
dbl
dbl
add
dbl
add
dbl
dbl
add
dbl
add
dbl
dbl
dbl
dbl
dbl
add
dbl
add
dbl
dbl
dbl
add
dbl
dbl
dbl
add
dbl
add
dbl
add
dbl
dbl
dbl
add
dbl
add
dbl
dbl
dbl
add
dbl
add
dbl
add
dbl
add


One can navigate the tree by indices, to get the second formula call of the second ECDH scalar multiplication:

In [10]:
action, subtree = tree.get_by_index([1, 0, 1])
print(repr(action))

FormulaAction(add[add-2007-bl], [Point([[X=383960763982903169689995079657542701951957535427344849026282264593569529481, Y=97816494580647658973605082832672362195798387552766223950530555908406646227345, Z=33112635526968736033324232912483052826450752467533070394363196286451437973911]] in EFDCoordinateModel("projective" on short Weierstrass curves)), Point([[X=24901324827761941171665406740352491860459252998228148257353674203781522895658, Y=1469593958564832835035565462147741560916652661670762870816949405978440504767, Z=42322585263999539773352738483857501665285392542933346948286130835216078214252]] in EFDCoordinateModel("projective" on short Weierstrass curves))]) = [Point([[X=106865350531828665272902459426695837837271776535211872763482234212642743419841, Y=102436521985123267638908548470272192599523323303084800549610360200950579480875, Z=14605232732661913809514868772836150792927623835013731687411437319958240413794]] in EFDCoordinateModel("projective" on short Weierstrass curves))]


The `[1,0,1]` path given above represented a walk through the execution trace, taking the second child of the root, then the first child then the second again.